In [1]:
import pytz
import datetime
import pandas as pd
import collections
import re
from pymongo import MongoClient
import numpy as np

In [4]:
def removeSpecialCharacters(value):
    return re.sub('[^A-Za-z0-9]+', ' ', str(value))

def cleanDataColumns(data):
    trueCol = []
    for col in data.columns:
        trueCol.append(removeSpecialCharacters(col))
    data.columns = trueCol
    return data

def limpa_cpf(cpfString):
    cpfString = cpfString.replace('.','')
    cpfString = cpfString.replace('-','')
    return int(cpfString)    

def limpeza_de_cpfs(dados, coluna):
    dados[coluna] = dados.loc[:,coluna].apply(limpa_cpf)
    return dados
    
def coluna_para_datetime(data, coluna):
    print('NECESSARIO UMA PROTECAO AQUI PARA CORRIGIR PROBLEMAS NOS DADOS')
    data[coluna] = pd.to_datetime(data[coluna], format='%d/%m/%Y')
    data[coluna] = data.loc[:,coluna].apply(date_to_Brazil_East_Utc)
    return data

def date_to_Brazil_East_Utc(datetime_value): # ele adianta o relogio em 3 horas
    local = pytz.timezone("Brazil/East")
    local_dt = local.localize(datetime_value, is_dst=None)
    utc_dt = local_dt.astimezone(pytz.utc)
    return utc_dt.strftime ("%Y-%m-%d %H:%M:%S")

def float_latin_nan_e_zero(entry):
    if _isNan(entry):
        return 0.0e0
    else:
        return floatLatin(entry)
    
def floatLatin(entry):
    if _isNan(entry):
        return np.nan
    entry = str(entry).replace('.','')
    entry = str(entry).replace(',','.')
    return float(entry)

def _isNan(value):
    if isinstance(value, str):
        return value == 'nan'
    else:
        return np.isnan(value)


# TRANSFORMACOES ESPECIFICAS DA TABELA ACORDOS

def transforma_Assessoria(valor):
    if str(valor) == 'nan':
        return 'SEMEAR'
    else:
        return str(valor)

def transforma_Cobrador(valor):
    if 'Semear_Renego' in valor:
        return 'Renegociacao'
    elif 'Multiparcelas' in valor:
        return 'Multiparcelas'
    else:
        return 'NULO'
        

# Leitura dos dados

In [5]:
arquivo_acordos = 'relatorio_de_acordos_012019_062019.csv'
acordos_dados = pd.read_csv(arquivo_acordos, sep=',', encoding='latin-1')
acordos_dados = cleanDataColumns(acordos_dados)
acordos_dados = coluna_para_datetime(acordos_dados, 'Data Opera o')
acordos_dados = limpeza_de_cpfs(acordos_dados, 'CPF CNPJ')

NECESSARIO UMA PROTECAO AQUI PARA CORRIGIR PROBLEMAS NOS DADOS


In [6]:
acordos_dados = coluna_para_datetime(acordos_dados, 'Pr x Vcto') # 'Proxima_data_de_pagamento'

NECESSARIO UMA PROTECAO AQUI PARA CORRIGIR PROBLEMAS NOS DADOS


In [7]:
acordos_dados.columns

Index(['Acordo', 'Agrupador', 'Cliente', 'Data Emiss o', 'Assessoria',
       'Pr x Vcto', 'Valor Total', 'Saldo Total', 'Situa o', 'Observa o',
       'Rating', 'CPF CNPJ', 'C digo do Cliente', 'Data Opera o',
       'Data Inclus o', 'Cobrador', 'Modalidade', 'Pr x Parc', 'Pr x Valor',
       'Saldo Atual', 'Usu rio', 'Taxa', 'Desconto', 'Parcelas',
       'Valor Entrada', 'Dias', 'Marcadores'],
      dtype='object')

In [10]:
collections.Counter(acordos_dados.loc[:,'Usu rio']).most_common()

[('COBRARR', 6365),
 ('MAR AZUL', 3832),
 ('Daniele Cristiane de Freitas Duarte', 3332),
 ('Bruno Naves de Oliveira', 2944),
 ('Anna Paula Pereira Gomes', 2679),
 ('Maria Fernanda Silva Pereira', 2642),
 ('Richard Nunes de Sousa', 2518),
 ('Lourrane Carolina de Jesus Segala', 2486),
 ('Raiane Lorena Soares Oliveira', 2129),
 ('Leonardo Pereira Vida', 1862),
 ('Madson Gabriel Rocha Silva', 1809),
 ('Trissia Gabrielle Fernandes Pereira Geronimo', 673),
 ('Leonilton Sarmento Batista', 523),
 ('Bruno Aparecido Silva de Jesus', 382),
 ('Mylena Soares dos Reis', 338),
 ('Silvana Cristina Freitas da Cunha', 294),
 ('Aline Alves Luiz', 281),
 ('Ingrid Pereira de Oliveira Silva', 273),
 ('Giovanna Mariana da Silva Santos', 265),
 ('Monaliza Nayanda Oliveira Rosa', 251),
 ('Gabriel Torres de Carvalho Timochenco', 154),
 ('Thailini Gomeri Barbosa', 122),
 ('Bruno Felipe Alves de Lima', 119),
 ('Ana Paula Azevedo', 21),
 ('MÃ´nia Alves de Souza', 1)]

# TRABALHANDO AQUI PARA DAR CARGA NO DATA POOL

In [191]:
valor_total = []
for i in acordos_dados.index:
    valor_total.append(float_latin_nan_e_zero(acordos_dados.loc[i,'Valor Entrada']))
    acordos_dict = {'Acordo_ID': int(acordos_dados.loc[i,'Acordo']),
                    'Numero_do_CPF': int(acordos_dados.loc[i,'CPF CNPJ']),
                    'Data_do_acordo': str(acordos_dados.loc[i,'Data Opera o']),
                    'Assessoria': transforma_Assessoria(acordos_dados.loc[i,'Assessoria']),
                    'Proxima_data_de_pagamento': str(acordos_dados.loc[i,'Pr x Vcto']),
                    'Valor_total_a_ser_pago': floatLatin(acordos_dados.loc[i,'Valor Total']),
                    'Modalidade_da_cobranca': transforma_Cobrador(acordos_dados.loc[i,'Cobrador']),
                    'Parcelas': int(acordos_dados.loc[i,'Parcelas']),
                    'Valor_de_entrada': float_latin_nan_e_zero(acordos_dados.loc[i,'Valor Entrada']),
                    'Rating_contabil_na_data_do_acordo': acordos_dados.loc[i,'Rating']
                    }
    if i > 10:
        break
    print(acordos_dict)

{'Acordo_ID': 10892, 'Numero_do_CPF': 48576786672, 'Data_do_acordo': '2019-04-24 03:00:00', 'Assessoria': 'COBRARR', 'Proxima_data_de_pagamento': '2019-05-02 03:00:00', 'Valor_total_a_ser_pago': 223.2, 'Modalidade_da_cobranca': 'Multiparcelas', 'Parcelas': 1, 'Valor_de_entrada': 0.0, 'Rating_contabil_na_data_do_acordo': 'A'}
{'Acordo_ID': 34018, 'Numero_do_CPF': 48576786672, 'Data_do_acordo': '2019-06-24 03:00:00', 'Assessoria': 'SEMEAR', 'Proxima_data_de_pagamento': '2019-06-24 03:00:00', 'Valor_total_a_ser_pago': 187.21, 'Modalidade_da_cobranca': 'Multiparcelas', 'Parcelas': 1, 'Valor_de_entrada': 0.0, 'Rating_contabil_na_data_do_acordo': 'A'}
{'Acordo_ID': 11554, 'Numero_do_CPF': 46426469149, 'Data_do_acordo': '2019-04-25 03:00:00', 'Assessoria': 'SEMEAR', 'Proxima_data_de_pagamento': '2019-04-26 03:00:00', 'Valor_total_a_ser_pago': 263.0, 'Modalidade_da_cobranca': 'Multiparcelas', 'Parcelas': 1, 'Valor_de_entrada': 0.0, 'Rating_contabil_na_data_do_acordo': 'A'}
{'Acordo_ID': 26451,

# Definicao da coluna acordos

In [71]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.cobrancas
acordos = db.acordos

# Carga no banco de dados

In [ ]:
for i in acordos_dados.index:
    acordos_dict = {'Acordo_ID': int(acordos_dados.loc[i,'Acordo']),
                    'Numero_do_CPF': int(acordos_dados.loc[i,'CPF CNPJ']),
                    'Data_do_acordo': str(acordos_dados.loc[i,'Data Opera o'])}
    print(acordos_dict)
    acordos.insert_one(acordos_dict)

# Todos os registros para um dado CPF

In [ ]:
cursor = acordos.find({'Numero_do_CPF': 54688795653})
for document in cursor:
    print(document)

# Todos os registros em um determinado intervalo de datas

In [93]:
start = date_to_Brazil_East_Utc(datetime.datetime(2019, 5, 3))
end = date_to_Brazil_East_Utc(datetime.datetime(2019, 6, 3))
query_datas_limite = {'Data_do_acordo': {'$lt': end, '$gte': start}}
cursor = acordos.find(query_datas_limite)
for document in cursor:
    print(document)

{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc08'), 'Acordo_ID': 21109, 'Numero_do_CPF': 8365434679, 'Data_do_acordo': '2019-05-20 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc0a'), 'Acordo_ID': 18267, 'Numero_do_CPF': 33604134153, 'Data_do_acordo': '2019-05-12 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc0b'), 'Acordo_ID': 19950, 'Numero_do_CPF': 33604134153, 'Data_do_acordo': '2019-05-16 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc10'), 'Acordo_ID': 23638, 'Numero_do_CPF': 52961753600, 'Data_do_acordo': '2019-05-28 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc1b'), 'Acordo_ID': 14261, 'Numero_do_CPF': 558598609, 'Data_do_acordo': '2019-05-03 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc1c'), 'Acordo_ID': 14263, 'Numero_do_CPF': 558598609, 'Data_do_acordo': '2019-05-03 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc1d'), 'Acordo_ID': 25165, 'Numero_do_CPF': 3089747600, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bc20'), '

{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd65'), 'Acordo_ID': 17813, 'Numero_do_CPF': 95936653668, 'Data_do_acordo': '2019-05-10 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd73'), 'Acordo_ID': 15520, 'Numero_do_CPF': 12559032104, 'Data_do_acordo': '2019-05-06 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd74'), 'Acordo_ID': 16835, 'Numero_do_CPF': 12559032104, 'Data_do_acordo': '2019-05-09 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd75'), 'Acordo_ID': 19398, 'Numero_do_CPF': 12559032104, 'Data_do_acordo': '2019-05-14 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd76'), 'Acordo_ID': 22684, 'Numero_do_CPF': 12559032104, 'Data_do_acordo': '2019-05-24 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd77'), 'Acordo_ID': 23375, 'Numero_do_CPF': 12559032104, 'Data_do_acordo': '2019-05-28 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd7f'), 'Acordo_ID': 17982, 'Numero_do_CPF': 12607825652, 'Data_do_acordo': '2019-05-10 03:00:00'}
{'_id': ObjectId('5d3f3ba55c7d6d0d62d0bd8

{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3be'), 'Acordo_ID': 15384, 'Numero_do_CPF': 4995407600, 'Data_do_acordo': '2019-05-06 03:00:00'}
{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3bf'), 'Acordo_ID': 24966, 'Numero_do_CPF': 4995407600, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3c0'), 'Acordo_ID': 24462, 'Numero_do_CPF': 94714347187, 'Data_do_acordo': '2019-05-30 03:00:00'}
{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3c3'), 'Acordo_ID': 18476, 'Numero_do_CPF': 93163061672, 'Data_do_acordo': '2019-05-13 03:00:00'}
{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3c6'), 'Acordo_ID': 22539, 'Numero_do_CPF': 98530585372, 'Data_do_acordo': '2019-05-24 03:00:00'}
{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3c7'), 'Acordo_ID': 22605, 'Numero_do_CPF': 98530585372, 'Data_do_acordo': '2019-05-24 03:00:00'}
{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3ca'), 'Acordo_ID': 21252, 'Numero_do_CPF': 9376489659, 'Data_do_acordo': '2019-05-21 03:00:00'}
{'_id': ObjectId('5d3f3bae5c7d6d0d62d0d3cb')

{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5b5'), 'Acordo_ID': 21691, 'Numero_do_CPF': 11063778697, 'Data_do_acordo': '2019-05-21 03:00:00'}
{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5bb'), 'Acordo_ID': 24701, 'Numero_do_CPF': 587064102, 'Data_do_acordo': '2019-05-30 03:00:00'}
{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5bc'), 'Acordo_ID': 15653, 'Numero_do_CPF': 4034588624, 'Data_do_acordo': '2019-05-06 03:00:00'}
{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5bd'), 'Acordo_ID': 15781, 'Numero_do_CPF': 1200880617, 'Data_do_acordo': '2019-05-07 03:00:00'}
{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5c0'), 'Acordo_ID': 16728, 'Numero_do_CPF': 12598056620, 'Data_do_acordo': '2019-05-08 03:00:00'}
{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5c4'), 'Acordo_ID': 15870, 'Numero_do_CPF': 15669141679, 'Data_do_acordo': '2019-05-07 03:00:00'}
{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5c6'), 'Acordo_ID': 18942, 'Numero_do_CPF': 5885802602, 'Data_do_acordo': '2019-05-14 03:00:00'}
{'_id': ObjectId('5d3f3bb55c7d6d0d62d0e5c7'), 

{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f790'), 'Acordo_ID': 21033, 'Numero_do_CPF': 26012049668, 'Data_do_acordo': '2019-05-20 03:00:00'}
{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f791'), 'Acordo_ID': 23376, 'Numero_do_CPF': 26012049668, 'Data_do_acordo': '2019-05-28 03:00:00'}
{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f794'), 'Acordo_ID': 20198, 'Numero_do_CPF': 6454681696, 'Data_do_acordo': '2019-05-16 03:00:00'}
{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f795'), 'Acordo_ID': 25236, 'Numero_do_CPF': 6454681696, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f798'), 'Acordo_ID': 14456, 'Numero_do_CPF': 84356081668, 'Data_do_acordo': '2019-05-03 03:00:00'}
{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f79a'), 'Acordo_ID': 20747, 'Numero_do_CPF': 4637930642, 'Data_do_acordo': '2019-05-17 03:00:00'}
{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f7a1'), 'Acordo_ID': 15677, 'Numero_do_CPF': 19159110600, 'Data_do_acordo': '2019-05-07 03:00:00'}
{'_id': ObjectId('5d3f3bbb5c7d6d0d62d0f7a4')

{'_id': ObjectId('5d3f3bc25c7d6d0d62d1090e'), 'Acordo_ID': 14839, 'Numero_do_CPF': 70195277104, 'Data_do_acordo': '2019-05-04 03:00:00'}
{'_id': ObjectId('5d3f3bc25c7d6d0d62d1090f'), 'Acordo_ID': 16315, 'Numero_do_CPF': 70195277104, 'Data_do_acordo': '2019-05-08 03:00:00'}
{'_id': ObjectId('5d3f3bc25c7d6d0d62d10912'), 'Acordo_ID': 20230, 'Numero_do_CPF': 43195326191, 'Data_do_acordo': '2019-05-17 03:00:00'}
{'_id': ObjectId('5d3f3bc25c7d6d0d62d10915'), 'Acordo_ID': 21602, 'Numero_do_CPF': 3211244999, 'Data_do_acordo': '2019-05-21 03:00:00'}
{'_id': ObjectId('5d3f3bc25c7d6d0d62d1091a'), 'Acordo_ID': 24311, 'Numero_do_CPF': 12362878600, 'Data_do_acordo': '2019-05-29 03:00:00'}
{'_id': ObjectId('5d3f3bc25c7d6d0d62d1091e'), 'Acordo_ID': 23898, 'Numero_do_CPF': 66672821015, 'Data_do_acordo': '2019-05-29 03:00:00'}
{'_id': ObjectId('5d3f3bc25c7d6d0d62d10924'), 'Acordo_ID': 14947, 'Numero_do_CPF': 27804360851, 'Data_do_acordo': '2019-05-05 03:00:00'}
{'_id': ObjectId('5d3f3bc25c7d6d0d62d10925

{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b4f'), 'Acordo_ID': 24513, 'Numero_do_CPF': 16013549672, 'Data_do_acordo': '2019-05-30 03:00:00'}
{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b54'), 'Acordo_ID': 14205, 'Numero_do_CPF': 47527030682, 'Data_do_acordo': '2019-05-03 03:00:00'}
{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b55'), 'Acordo_ID': 14206, 'Numero_do_CPF': 47527030682, 'Data_do_acordo': '2019-05-03 03:00:00'}
{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b56'), 'Acordo_ID': 19557, 'Numero_do_CPF': 47527030682, 'Data_do_acordo': '2019-05-15 03:00:00'}
{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b57'), 'Acordo_ID': 20041, 'Numero_do_CPF': 47527030682, 'Data_do_acordo': '2019-05-16 03:00:00'}
{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b58'), 'Acordo_ID': 20865, 'Numero_do_CPF': 47527030682, 'Data_do_acordo': '2019-05-19 03:00:00'}
{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b59'), 'Acordo_ID': 22408, 'Numero_do_CPF': 47527030682, 'Data_do_acordo': '2019-05-24 03:00:00'}
{'_id': ObjectId('5d3f3bca5c7d6d0d62d11b5

{'_id': ObjectId('5d3f3bd05c7d6d0d62d12ccc'), 'Acordo_ID': 23388, 'Numero_do_CPF': 11459572696, 'Data_do_acordo': '2019-05-28 03:00:00'}
{'_id': ObjectId('5d3f3bd05c7d6d0d62d12cd2'), 'Acordo_ID': 23052, 'Numero_do_CPF': 1846209021, 'Data_do_acordo': '2019-05-27 03:00:00'}
{'_id': ObjectId('5d3f3bd05c7d6d0d62d12cd4'), 'Acordo_ID': 20689, 'Numero_do_CPF': 4237966151, 'Data_do_acordo': '2019-05-17 03:00:00'}
{'_id': ObjectId('5d3f3bd05c7d6d0d62d12cd8'), 'Acordo_ID': 21850, 'Numero_do_CPF': 6595121654, 'Data_do_acordo': '2019-05-22 03:00:00'}
{'_id': ObjectId('5d3f3bd05c7d6d0d62d12cd9'), 'Acordo_ID': 25798, 'Numero_do_CPF': 6595121654, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bd05c7d6d0d62d12cda'), 'Acordo_ID': 25801, 'Numero_do_CPF': 6595121654, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bd05c7d6d0d62d12cdb'), 'Acordo_ID': 25809, 'Numero_do_CPF': 6595121654, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bd05c7d6d0d62d12ce3'), '

{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e01'), 'Acordo_ID': 21720, 'Numero_do_CPF': 97202207672, 'Data_do_acordo': '2019-05-22 03:00:00'}
{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e04'), 'Acordo_ID': 25076, 'Numero_do_CPF': 11836989172, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e06'), 'Acordo_ID': 17146, 'Numero_do_CPF': 64920933134, 'Data_do_acordo': '2019-05-09 03:00:00'}
{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e07'), 'Acordo_ID': 18026, 'Numero_do_CPF': 3470160651, 'Data_do_acordo': '2019-05-10 03:00:00'}
{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e09'), 'Acordo_ID': 25066, 'Numero_do_CPF': 3590829125, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e0a'), 'Acordo_ID': 25067, 'Numero_do_CPF': 3590829125, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e0b'), 'Acordo_ID': 25068, 'Numero_do_CPF': 3590829125, 'Data_do_acordo': '2019-05-31 03:00:00'}
{'_id': ObjectId('5d3f3bd65c7d6d0d62d13e14'),